In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install dtreeviz

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import random

from dtreeviz.trees import *


from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import f1_score, accuracy_score, precision_score, roc_auc_score
from sklearn import tree


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


1. Upload data set
2. data analysis
3. clean the data
4. feature selection
5. 
 

In [23]:
my_data = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')
my_data.reset_index(drop=True, inplace=True)
# my_data.drop(['Date'],inplace=True,axis=1)
my_data

In [24]:
print(my_data.dtypes)
my_data.describe()

In [25]:
print("number of missing values for each feature:")
print(my_data.isna().sum())
print("\nnumber of duplicate raws in the data set:")
print(my_data.duplicated().sum())

In [26]:
my_data.drop_duplicates(inplace=True)
my_data.dropna(inplace=True)

print("number of missing values for each feature:")
print(my_data.isna().sum())
print("\nnumber of duplicate raws in the data set:")
print(my_data.duplicated().sum())
my_data.shape

In [27]:
my_data['Date'] = my_data['Date'].apply(lambda x: int(x.split('-')[1]))
my_data['Date']

In [28]:
numerical= my_data.drop(['RainTomorrow'], axis=1).select_dtypes('number').columns

categorical = my_data.select_dtypes('object').columns

print(f'Numerical Columns:  {my_data[numerical].columns}')
print('\n')
print(f'Categorical Columns: {my_data[categorical].columns}')

In [29]:
pd.options.plotting.backend = "plotly"
my_data[numerical].plot(kind='hist')

In [30]:
my_data[categorical].plot(kind='hist')

In [31]:
#define number of rows and columns for subplots
nrow=5
ncol=5

# make a list of all dataframes 
fig, axes = plt.subplots(nrow, ncol)
fig.set_size_inches(40, 40)
fig.set_dpi(100)
# columns= []
# color_map = [[{i: random.random()} for i in range(33)] for j in range(12)]
# my_colors = [(x/10.0, x/100.0, x/10.0) for x in range(10)]

my_colors = [ (random.random(),random.random(),0.75) for I  in range(100)]



# plot counter
for inx,col in enumerate(my_data.columns):
    axes[inx%nrow,inx//nrow].set_title(col)
    axes[inx%nrow,inx//nrow].bar(my_data[col].unique(),height=my_data[col].value_counts(),color=my_colors)


In [32]:
male_heart = my_data[(my_data['RainTomorrow'] == 'Yes') & (my_data['RainToday'] == 'Yes')]['RainToday'].count()
female_heart = my_data[(my_data['RainTomorrow'] == 'Yes') & (my_data['RainToday'] == 'No')]['RainToday'].count()

male_normal = my_data[(my_data['RainTomorrow'] != 'Yes') & (my_data['RainToday'] == 'Yes')]['RainToday'].count()
female_normal = my_data[(my_data['RainTomorrow'] != 'Yes') & (my_data['RainToday'] == 'No')]['RainToday'].count()


heart_df = pd.DataFrame({'RainTomorrow':[male_normal,male_heart],'RainToday':[0,1]})
norm_df = pd.DataFrame({'RainTomorrow':[female_normal,female_heart],'RainToday':[0,1]})


heart = [male_heart,female_heart]
target = [male_normal,female_normal]
index = ['Yes','No']
df = pd.DataFrame({'RainToday': target,
                   'RainTomorrow': heart}, index=index)

df.plot.bar()


In [33]:
fig=plt.figure(figsize=(20,8),facecolor='white')

plt.suptitle('Relationship between continuous variables and one categorical variable',fontsize="xx-large",fontweight="bold")

gs=fig.add_gridspec(1,2)


ax=[_ for i in range(2)]

ax[0]=fig.add_subplot(gs[0,0])
ax[1]=fig.add_subplot(gs[0,1])


for i in range(2):
    ax[i].set_yticklabels([])
    ax[i].tick_params(axis='y',length=0)
    
    for direction in ["top","right", 'left']:
        ax[i].spines[direction].set_visible(False)
    
sb.lineplot(data=my_data,x='Sunshine',y='Evaporation',ax=ax[0],hue='RainTomorrow')
sb.scatterplot(data=my_data,x='Cloud3pm',y='Rainfall',hue='RainTomorrow',ax=ax[1],palette='viridis',markers='o',alpha=0.1)



In [34]:
my_data.rename(columns={'RainTomorrow':'Target'},inplace=True)
my_data

In [35]:
le = preprocessing.LabelEncoder()
for col in my_data.columns:
    if my_data[col].dtypes == object:
        my_data[col] = le.fit_transform(my_data[col])
my_data

In [36]:

fig, ax = plt.subplots(figsize=(20,20))
mask = np.triu(np.ones_like(my_data.iloc[:,:-1].corr(), dtype=bool))
sb.heatmap(my_data.iloc[:,:-1].corr(),mask=mask, annot=True, linewidths=.5, ax=ax)


In [37]:
# corr = my_data.iloc[:,:-1].corr()
# my_data.iloc[:,:-1].corr()[abs(corr)>=0.5]

# Create correlation matrix
corr_matrix = corr = my_data.iloc[:,:-1].corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]


# Drop features 
my_data.drop(to_drop, axis=1, inplace=True)
my_data



In [38]:
fig, ax = plt.subplots(figsize=(12,12))
sb.heatmap(my_data.corr()[:-1][['Target']].sort_values('Target').tail(11),
vmax=1, vmin=-1, cmap='YlGnBu', annot=True, ax=ax);
ax.invert_yaxis()

In [39]:
def get_mapper(model,X):
    mapper = dict()
    for key,val,_,_ in zip(X.columns[model.tree_.feature], model.tree_.threshold, 
                           model.tree_.children_left, model.tree_.children_right):
        mapper[key]= val
    return mapper

In [40]:
def run__eval_models(my_data,split_size=50,max_depth=5):

    #split between features and traget variables 
    X = my_data.iloc[:,:-1]
    y = my_data.iloc[:,-1]

    alpah_lst = np.arange(0.1,0.9,0.2)
    iter_num = 1
    model = DecisionTreeClassifier(criterion="gini",splitter="best",max_depth=max_depth,min_samples_split=split_size)


    nrow=2
    ncol=2

    # make a list of all dataframes 
    fig, axes = plt.subplots(nrow, ncol)
    fig.set_size_inches(18.5, 10.5)
    fig.set_dpi(100)
    columns= ['precision','accuracy','f1','RoC']


    #Use Stratified K fold to split train-test data
    for alpha in alpah_lst:

        model_accuracy = []
        model_precision = []
        model_f1 = []
        model_Roc = []


        new_model_accuracy = []
        new_model_precision = []
        new_model_f1 = []
        new_model_Roc = []
        print(alpha)

        skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            model.fit(X_train,y_train)
            y_pred = model.predict(X_test)
            model_accuracy.append(accuracy_score( y_test,y_pred))
            model_precision.append(precision_score(y_test, y_pred))
            model_f1.append(f1_score(y_test,y_pred))
            model_Roc.append(roc_auc_score(y_test,y_pred))


            mapper = get_mapper(model,X_train)
            y_pred = np.zeros((iter_num, *y_pred.shape))
            for index in range(iter_num):
                tmp = X_test.copy()
                for r_idx,row in X_test.iterrows():
                    for c_idx,feature_val in enumerate(row):
                        feature = tmp.columns[c_idx]
                        if feature in mapper and np.random.uniform(low=0.0, high=1.0, size=None) < alpha:
                            if tmp.at[r_idx,feature] > mapper[feature]:
                                tmp.at[r_idx,feature] = mapper[feature] * 1.01 if mapper[feature] < 0 else  mapper[feature] / 1.01
                            else: 
                                tmp.at[r_idx,feature] = mapper[feature] / 1.01 if mapper[feature] < 0 else  mapper[feature] * 1.01
                y_pred[index] = model.predict(tmp)
            y_pred = np.median(y_pred,axis=0).astype(int)
            new_model_accuracy.append(accuracy_score(y_test,y_pred))
            new_model_precision.append(precision_score(y_test, y_pred))
            new_model_f1.append(f1_score(y_test, y_pred))
            new_model_Roc.append(roc_auc_score(y_test, y_pred))


        axes[0,0].set_title('precision')
        axes[0,0].plot(new_model_precision,'--',label=str(alpha))
        axes[0,0].legend()

        axes[0,1].set_title('accuracy')
        axes[0,1].plot(new_model_accuracy,'--',label=str(alpha))
        axes[0,1].legend()

        axes[1,0].set_title('f1')
        axes[1,0].plot(new_model_f1,'--',label=str(alpha))
        axes[1,0].legend()

        axes[1,1].set_title('Roc')
        axes[1,1].plot(new_model_Roc,'--',label=str(alpha))
        axes[1,1].legend()


    #     #print(model_accuracy,model_precision)    
    #     print(f"mean accuracy: {np.asarray(model_accuracy).mean()}")
    #     print(f"mean precision: {np.asarray(model_precision).mean()}")
    #     print("-"*30)
    #     #print(new_model_accuracy,new_model_precision)    
    #     print(f"mean accuracy: {np.asarray(new_model_accuracy).mean()}")
    #     print(f"mean precision: {np.asarray(new_model_precision).mean()}")
    #     print("="*30)
    #     print(model_accuracy)

    axes[0,0].plot(model_precision)
    axes[0,1].plot(model_accuracy)
    axes[1,0].plot(model_f1)
    axes[1,1].plot(model_Roc)
    
    return model


In [41]:
np.random.seed(10)
model = run__eval_models(my_data)

In [42]:
X = my_data.iloc[:,:-1]
y = my_data.iloc[:,-1]

viz = dtreeviz(model, X, y,
                target_name="Target",
                feature_names=my_data.columns,
                class_names=['0','1'])

viz

In [43]:
# my_data.drop(['ever_married','gender','Residence_type'],inplace= True,axis=1)
# np.random.seed(10)
# model = run__eval_models(my_data)